<a href="https://colab.research.google.com/github/siren7075/OpenAI_Embeddings_Public/blob/main/OpenAI_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Open AI and Import Data

In [20]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [89]:
# imports
from typing import List

import pandas as pd
import pickle
import ast

from openai.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
)

In [26]:
openai.api_key="sk-"

# Get Embeddings of 1000 bonds

In [27]:
df = pd.read_csv("/content/data.refined_combined.csv")
df.head(5)

,cusip,name,instrument,sector,industry,subindustry,currency,country,coupon,maturity,rating,issuer,liquidity,bids,asks,combined
0,00184AAC9,AOL TIME WARNER-GLOBAL,TXBLIB,CORPORATE,INDUSTRIAL,MEDIA_ENTERTAINMENT,USD,UNITED STATES,7.625,04/15/2031,BAA3,TWX,6.0,NaN,NaN,"7.625,47953,BAA3,TWX,USD,UNITED STATES"
1,02079KAB3,ALPHABET INC,TXBLIB,CORPORATE,INDUSTRIAL,TECHNOLOGY,USD,UNITED STATES,3.375,02/25/2024,AA1,GOOGL,101.0,1.0,NaN,"3.375,45347,AA1,GOOGL,USD,UNITED STATES"
2,02079KAC1,ALPHABET INC,TXBLIB,CORPORATE,INDUSTRIAL,TECHNOLOGY,USD,UNITED STATES,1.998,08/15/2026,AA1,GOOGL,110.0,NaN,2.0,"1.998,46249,AA1,GOOGL,USD,UNITED STATES"
3,02079KAE7,ALPHABET INC,TXBLIB,CORPORATE,INDUSTRIAL,TECHNOLOGY,USD,UNITED STATES,1.900,08/15/2040,AA1,GOOGL,37.0,3.0,1.0,"1.9,51363,AA1,GOOGL,USD,UNITED STATES"
4,02079KAF4,ALPHABET INC,TXBLIB,CORPORATE,INDUSTRIAL,TECHNOLOGY,USD,UNITED STATES,2.050,08/15/2050,AA1,GOOGL,112.0,4.0,5.0,"2.05,55015,AA1,GOOGL,USD,UNITED STATES"


In [28]:
len(df)

789

In [30]:
df_1000 = df.head(1000)
len(df_1000)

789

In [31]:
df['davinci_similarity'] = df_1000.combined.apply(lambda x: get_embedding(x, engine='text-similarity-ada-001'))
df.to_csv('/Cusips_Embeddings_refined.csv')

In [43]:
df_embeddings = pd.read_csv("/Cusips_Embeddings_refined.csv")
df_embeddings_1000 = df_embeddings.head(1000)

In [44]:
# get embeddings for all strings
embeddings = df_embeddings_1000.davinci_similarity
# remove colons in the list
embeddings_clean = [ast.literal_eval(i) for i in embeddings]

In [45]:
embeddings_clean[0][0:3]

[0.001978249754756689, 0.026390312239527702, 0.013666046783328056]

# Get distances and indices of nearest neighbors

In [75]:
# get the source embedding
query_embedding = embeddings_clean[224]

In [76]:
# get distances between the source embedding and other embeddings (function from embeddings_utils.py)
distances = distances_from_embeddings(query_embedding, embeddings_clean, distance_metric="cosine")
print (distances[0:3])

[0.07535786964546454, 0.07352040489687195, 0.09049253363570997]


In [77]:
# get indices of nearest neighbors (function from embeddings_utils.py)
indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances)
print (indices_of_nearest_neighbors[0:3])

[224 223 580]


#Bond Recommendation based on similarity score

---



In [78]:
combined_strings = df_embeddings_1000.combined.values.tolist()
# get the source string
query_string = combined_strings[224]
print(f"Source string: {query_string}")

Source string: 4.75,46767,BAA2,HPQ,USD,UNITED STATES


In [100]:
nameList = df_embeddings_1000.name.values.tolist()
liquidityList = df_embeddings_1000.liquidity.values.tolist()
bidList = df_embeddings_1000.bids.values.tolist()
askList = df_embeddings_1000.asks.values.tolist()
industryList = df_embeddings_1000.subindustry.values.tolist()
nameList[224]

'HP INC'

# Recommend similar bond for HP INC
Source String: 4.75,46767,BAA2,HPQ,USD,UNITED STATES

In [103]:
# print out its k nearest neighbors, defaults to 5
k_counter = 0
for i in indices_of_nearest_neighbors:
# skip any strings that are identical matches to the starting string
    if query_string == combined_strings[i]:
        continue
    if industryList[224] != industryList[i]:
        continue
    # stop after printing out k articles
    if k_counter >= 5:
        break
    k_counter += 1

    # print out the similar strings and their distances
    print(
        f"""
    --- Recommendation #{k_counter} (nearest neighbor {k_counter} of 5 ---
    String: {combined_strings[i]}
    Security Name: {nameList[i]}
    Accesibility: Trace: {liquidityList[i]}, Bids: {bidList[i]}, Asks: {askList[i]}
    Index: {i}
    Excel Line: {i+2}
    Distance: {distances[i]:0.3f}"""
    )


    --- Recommendation #1 (nearest neighbor 1 of 5 ---
    String: 4,47223,BAA2,HPQ,USD,UNITED STATES
    Security Name: HP INC
    Accesibility: Trace: 141.0, Bids: 2.0, Asks: 3.0
    Index: 223
    Excel Line: 225
    Distance: 0.025

    --- Recommendation #2 (nearest neighbor 2 of 5 ---
    String: 4.75,47178,B2,HPQ,USD,UNITED STATES
    Security Name: PLANTRONICS INC
    Accesibility: Trace: nan, Bids: nan, Asks: nan
    Index: 580
    Excel Line: 582
    Distance: 0.025

    --- Recommendation #3 (nearest neighbor 3 of 5 ---
    String: 4.75,46583,BAA2,VRSN,USD,UNITED STATES
    Security Name: VERISIGN INC
    Accesibility: Trace: 135.0, Bids: 3.0, Asks: 1.0
    Index: 635
    Excel Line: 637
    Distance: 0.026

    --- Recommendation #4 (nearest neighbor 4 of 5 ---
    String: 4.75,45627,BAA3,TRMB,USD,UNITED STATES
    Security Name: TRIMBLE NAVIGATION LTD
    Accesibility: Trace: 34.0, Bids: 1.0, Asks: nan
    Index: 631
    Excel Line: 633
    Distance: 0.028

    --- Recomm